# <font color=green>LivingEarth PNG - FAO LCCS Level 4</font>

### An environmental layers testing framework for the FAO land cover classification system

The purpose of this notebook is to provide an easy-to-use method for testing environmental layers to use for classification and seeing how changes to particular layers effect the final Land Cover Classification. You can easily test with different environmental layer inputs, and different locations. 

This code defines 5 variables to contain the binary layers required to reach a level 3 classification:
1. <font color=blue>**vegetat_veg_cat:**</font> Vegetated / Non-Vegetated 
2. <font color=blue>**aquatic_wat_cat:**</font> Aquatic / Terrestrial 
3. <font color=blue>**cultman_agr_cat:**</font> Natural Vegetation / Crop or Managed Vegetation
4. <font color=blue>**artific_urb_cat:**</font> Natural Surfaces / Artificial Surfaces
5. <font color=blue>**artwatr_wat_cat:**</font> Natural Water / Artificial Water

Shallow level 4
- deriving woody/non wood using WCF
- pulling out mangrove, supratidal forests (STF) and saltmarsh - the latter two using DEM < 10 m


#### Install the environment (need DEA fc and wofs packages) using the following on command line

In [ ]:
'''
python3 -m venv ~/venvs/VP_Environment
source ~/venvs/VP_Environment/bin/activate
deactivate
realpath /env/lib/python3.10/site-packages > ~/venvs/VP_Environment/lib/python3.10/site-packages/base_venv.pth
source ~/venvs/VP_Environment/bin/activate
pip install --extra-index-url=https://packages.dea.ga.gov.au/ fc
pip install ephem
pip install --index-url https://packages.dea.ga.gov.au/ wofs
python -m ipykernel install --user --name=VP_Environment
cd venvs/VP_Environment
touch setup.py

##### add in text below to setup.py

from setuptools import setup, find_packages

setup(
    name='VP_Environment',
    version='1.0',
    packages=find_packages(),
    install_requires=[
        'ephem',
        'wofs',
        'fc'
    ],
)

#####

python setup.py bdist_egg

'''

### **Import modules**

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import os, sys
import copy

import numpy as np
import xarray as xr
import geopandas as gpd
import rasterio

from matplotlib import pyplot
import matplotlib.pyplot as plt

import datacube
from datacube.utils import masking
from datacube.utils.geometry import CRS
from datacube.testutils.io import rio_slurp_xarray
from dea_tools.plotting import display_map
from dea_tools.spatial import xr_rasterize

#import le_lccs modules
sys.path.append("../../livingearth_lccs")
from le_lccs.le_ingest import gridded_ingest
from le_lccs.le_classification import lccs_l3
from le_lccs.le_classification import lccs_l4

# for virtual products
sys.path.insert(1, "/home/jovyan/code/livingearth_png/le_plugins")
import yaml
import importlib
from datacube.virtual import catalog_from_file
from datacube.virtual import DEFAULT_RESOLVER

# outputs
from datacube.utils.cog import write_cog
from datacube.drivers.netcdf import write_dataset_to_netcdf

dc = datacube.Datacube(app="level3")

# virtual product catalog
catalog = catalog_from_file('../le_plugins/virtual_product_cat.yaml')

#### Set Up Dask

In [ ]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()
client = Client(cluster)
client

#### EASI Cache

Don't use this for now as causes problems with some files on the S3 bucket. Once fixed this can be removed.

In [ ]:
from os import environ
environ["AWS_HTTPS"] = "YES"
environ["GDAL_HTTP_PROXY"] = ""
print(f'Will use caching proxy at: {environ.get("GDAL_HTTP_PROXY")}')

In [ ]:
sys.path.append("/home/jovyan/code/easi-notebooks/")
from easi_tools import EasiDefaults, notebook_utils

easi = EasiDefaults()
                
notebook_utils.localcluster_dashboard(client=client, server=easi.hub)

#### AWS Access

In [ ]:
## Optional: Access AWS "requester-pays" buckets
# This is necessary for Landsat ("landsatN_c2l2_*") and Sentinel-2 ("s2_l2a") products
from datacube.utils.aws import configure_s3_access
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client) ### add client=client here if launching a dask cluster

#### display AOI

In [ ]:
#latitude = (-7.329, -7.339)
#longitude = (144.44, 144.45)

# Western Gulf mangrove area
latitude = (-7.3, -7.4)
longitude = (144.4, 144.5)

# # Port Moresby
#latitude = (-9.4, -9.5)
#longitude = (147.1, 147.2)

display_map(longitude, latitude)

## <font color=blue>Set up query</font>

In [ ]:
time = ('2020-01-01', '2020-12-31')

crs = "EPSG:32755"
res = (30, -30)

query =({'time': time,
             'latitude':latitude,
            'longitude':longitude,
            'output_crs':crs,
            'resolution':res,
            'dask_chunks' : {'time' : 10, 'x' : 2048, 'y' : 2048}})

## <font color=blue>Create environmental layers for Level 3</font>

### 1. Vegetated / Non-Vegetated

   * **Primarily Vegetated Areas**:
   This class applies to areas that have a vegetative cover of at least 4% for at least two months of the year, consisting of Woody (Trees, Shrubs) and/or Herbaceous (Forbs, Graminoids) lifeforms, or at least 25% cover of Lichens/Mosses when other life forms are absent.
      
   * **Primarily Non-Vegetated Areas**:
   Areas which are not primarily vegetated.
 

Fractional cover (FC) is used to distinguish between vegetated and not vegetated. 
http://data.auscover.org.au/xwiki/bin/view/Product+pages/Landsat+Fractional+Cover
<br>We are using the 90th annual percentile for both Photosyntheic (PV) and Non-photosynthetic (NPV) vegetation. This removes noise and outliers and gives a robust maximum annual value. A threshold is then applied where PV or NPV is greater than 50%, the rationale being that if a pixel is greater than 50% PV or NPV we can be confident that it is likely to be vegetated. In addition, a maximum threshold value is given to NPV as non-photosynthetic vegetation and bare soil (BS) fractions can be unreliable at maximum values due to inherent issues with unmixing NPV and BS signatures.

<font color=red>**TODO:**</font> need to calculate number of observations for annual time series to define vegetation correctly as in FAO guidelines. This will likely be similar to WOfS wet/clear obervations but for FC where PV or NPV is greater than 50% for 60 days per year

In [ ]:
# Load Fractional Cover

# Need to add any tranformations for the VP you're using
# Get location of transformation
transformation = "fractional_cover"
trans_loc = importlib.import_module(transformation)
trans_class = transformation.split('.')[-1]

DEFAULT_RESOLVER.register('transform', trans_class, getattr(trans_loc, trans_class) )

In [ ]:
product = catalog['fractional_cover']
fractional_cover = product.load(dc, **query)
fractional_cover = masking.mask_invalid_data(fractional_cover)

In [ ]:
# Need to compute fractional cover here as needed as a mask later
fractional_cover = fractional_cover.compute()

In [ ]:
# Load WOfS

# Need to add any tranformations for the VP you're using
# Get location of transformation
transformation = "WOfS"
trans_loc = importlib.import_module(transformation)
trans_class = transformation.split('.')[-1]

DEFAULT_RESOLVER.register('transform', trans_class, getattr(trans_loc, trans_class) )

In [ ]:
product = catalog['WOfS']
wofs = product.load(dc, **query)
wofs = masking.mask_invalid_data(wofs)
wofs_mask = wofs["frequency"] >= 0.2

In [ ]:
# Need to compute WOfS here as needed as a mask later
wofs_mask = wofs_mask.compute()

In [ ]:
# Create binary layer representing vegetated (1) and non-vegetated (0)
vegetat = ((fractional_cover["PV_PC_90"] > 25).fillna(0) - (fractional_cover["NPV_PC_90"] > 25).fillna(0))
vegetat = (vegetat.where(vegetat>0)*0+1).fillna(0)

# mask out water here
vegetat = vegetat.where(wofs_mask == 0, 0, 1)

# Convert to Dataset and add name
vegetat_veg_cat_ds = vegetat.to_dataset(name="vegetat_veg_cat")#.squeeze().drop('time')

In [ ]:
vegetat_veg_cat_ds.vegetat_veg_cat.plot()

### 2. Aquatic / Terrestrial

   * **Primarily Vegetated, Terrestrial**: The vegetation is influenced by the edaphic substratum
   * **Primarily Non-Vegetated, Terrestrial**: The cover is influenced by the edaphic substratum
   * **Primarily Vegetated, Aquatic or regularly flooded**: The environment is significantly influenced by the presence of water over extensive periods of time. The water is the dominant factor determining natural soil development and the type of plant communities living on its surface
   * **Primarily Non-Vegetated, Aquatic or regularly flooded**: Permanent or regularly flood aquatic areas
   

Water Observations from Space (WOfS) is used to distinguish aquatic and terrestrial areas.
https://www.sciencedirect.com/science/article/pii/S0034425715301929?via%3Dihub
<br> A threshold of 20% is applied for the annual summary dataset to remove flood events not indicative of the landscape.
<br>The Mangrove layer are also used for relevant coastal landscapes.

<font color=red>**TODO:**</font> need to fix the issue of clouds in WOFS, which results in zeros in lccs map 

In [ ]:
# note: wofs (loaded in level 1 as wofs_mask)

# load mangroves as mask
# gmw_2020 = '../data/GMW/gmw_v3_2020_vec.shp'
gmw_2020 = "s3://oa-bluecarbon-work-easi/livingearth-png/gmw_v3_2020_vec_png.gpkg"

# load in mangrove vector data just for AOI extent
bbox = [longitude[0], latitude[1], longitude[1], latitude[0]]

# Load the mangrove vector data within the AOI extent
gmw = gpd.read_file(gmw_2020, bbox=bbox)

# Check if the mangrove vector dataset is empty
if gmw.empty:
    # If the mangrove vector dataset is empty, create a raster of zeros matching the shape of the WOfS mask
    mangrove = xr.DataArray(np.zeros_like(wofs_mask), coords=wofs_mask.coords, dims=wofs_mask.dims, attrs=wofs_mask.attrs)
else:
    # If the mangrove vector dataset is not empty, rasterize it to match the shape of the WOfS mask
    # Get the bounding box coordinates
    xmin, ymin, xmax, ymax = bbox
    
    # Get the mangrove vector data within the AOI extent
    gmw_aoi = gmw.cx[xmin:xmax, ymin:ymax]
    
    # Rasterize the mangrove vector data to match the shape of the WOfS mask
    mangrove = xr_rasterize(gdf=gmw_aoi, da=wofs_mask)


In [ ]:
# Open Murray's tidal wetland probability (2017-2019) file as xarray
# tidal_wetland_file = '../data/Tidal_wetland_Murray_20172019_30m_PNG.tif'
tidal_wetland_file = "s3://oa-bluecarbon-work-easi/livingearth-png/Tidal_wetland_Murray_20172019_30m_PNG.tif"
tidal_wetland = rio_slurp_xarray(tidal_wetland_file, gbox=vegetat_veg_cat_ds.geobox)

# Threshold probability layer to 50%
tidal_wetland_extent = ((tidal_wetland.where(tidal_wetland>50))*0+1).fillna(0)

# Remove mudflats
tidal_wetland_veg = vegetat_veg_cat_ds.vegetat_veg_cat * tidal_wetland_extent

In [ ]:
# Create binary layer representing aquatic (1) and terrestrial (0)

# For coastal landscapes use the following
aquatic_wat = (wofs_mask + mangrove + tidal_wetland_veg)
aquatic_wat = (aquatic_wat.where(aquatic_wat > 0)*0+1).fillna(0)

# Convert to Dataset and add name
aquatic_wat_cat_ds = aquatic_wat.to_dataset(name="aquatic_wat_cat")#.squeeze().drop('time')             

In [ ]:
# Plot output
aquatic_wat_cat_ds["aquatic_wat_cat"].plot.imshow(figsize=(6, 5))

### 3. Natural Vegetation / Crop or Managed Vegetation

   * **Primarily Vegetated, Terrestrial, Artificial/Managed**: Cultivated and Managed Terrestrial Areas
   * **Primarily Vegetated, Terrestrial, (Semi-)natural**: Natural and Semi-Natural Vegetation
   * **Primarily Vegetated, Aquatic or Regularly Flooded, Artificial/Managed**: Cultivated Aquatic or Regularly Flooded Areas
   * **Primarily Vegetated, Aquatic or Regularly Flooded, (Semi-)natural**: Natural and Semi-Natural Aquatic or Regularly Flooded Vegetation

#### Excluding cultivated areas

In [ ]:
# Create a raster of zeros
cultman = xr.DataArray(np.zeros_like(wofs_mask), 
                       coords=wofs_mask.coords, dims=wofs_mask.dims, attrs=wofs_mask.attrs)

# Convert to Dataset and add name
cultman_agr_cat_ds = cultman.to_dataset(name="cultman_agr_cat")

#### Test based on Landsat geomedian NDVI

In [ ]:
# # Need to add any tranformations for the VP you're using
# # Get location of transformation
# transformation = "geomedian"
# trans_loc = importlib.import_module(transformation)
# trans_class = transformation.split('.')[-1]

# DEFAULT_RESOLVER.register('transform', trans_class, getattr(trans_loc, trans_class) )

In [ ]:
# # load geomedian
# product = catalog['geomedian']
# geomedian = product.load(dc, **query)

In [ ]:
# # Create binary layer representing cultivated (1) and natural (0)

# # calculate NDVI (PLACEHOLDER)
# NDVI = (geomedian.nbart_nir-geomedian.nbart_red)/(geomedian.nbart_nir+geomedian.nbart_red)
# cultman = (NDVI >= 0.4) & (NDVI <= 0.7)

# # Convert to Dataset and add name
# cultman_agr_cat_ds = cultman.to_dataset(name="cultman_agr_cat")#.squeeze().drop('time')

In [ ]:
# Plot output
cultman_agr_cat_ds["cultman_agr_cat"].plot.imshow(figsize=(6, 5))

### 4. Natural Surfaces / Artificial Surfaces

In [ ]:
OSM = "s3://oa-bluecarbon-work-easi/livingearth-png/papua-new-guinea.gpkg"

# load in OSM vector data just for AOI extent
bbox = [longitude[0], latitude[1], longitude[1], latitude[0]]

OSM_blds = gpd.read_file(OSM, layer='buildings', bbox=bbox)
OSM_airports = gpd.read_file(OSM, layer='aeroway_ln', bbox=bbox)
OSM_roads = gpd.read_file(OSM, layer='highway_ln', bbox=bbox)

In [ ]:
# get bbox to get geom of gdf
xmin, ymin, xmax, ymax = bbox

In [ ]:
# Check if the vector dataset is empty
if OSM_blds.empty:
    # If the vector dataset is empty, create a raster of zeros matching the shape of the geomedians
    OSM_blds_xr = xr.DataArray(
        np.zeros_like(wofs_mask),
        coords=wofs_mask.coords,
        dims=wofs_mask.dims,
        attrs=wofs_mask.attrs,
    )
else:
    OSM_blds_AOI = OSM_blds.cx[xmin:xmax, ymin:ymax]
    OSM_blds_xr = xr_rasterize(gdf=OSM_blds_AOI, da=wofs_mask)

# Check if the vector dataset is empty
if OSM_airports.empty:
    # If the vector dataset is empty, create a raster of zeros matching the shape of the geomedians
    OSM_airports_xr = xr.DataArray(
        np.zeros_like(wofs_mask),
        coords=wofs_mask.coords,
        dims=wofs_mask.dims,
        attrs=wofs_mask.attrs,
    )
else:
    OSM_airports_AOI = OSM_airports.cx[xmin:xmax, ymin:ymax]
    OSM_airports_xr = xr_rasterize(gdf=OSM_airports_AOI, da=wofs_mask)

    # Check if the vector dataset is empty
if OSM_roads.empty:
    # If the vector dataset is empty, create a raster of zeros matching the shape of the geomedians
    OSM_roads_xr = xr.DataArray(
        np.zeros_like(wofs_mask),
        coords=wofs_mask.coords,
        dims=wofs_mask.dims,
        attrs=wofs_mask.attrs,
    )
else:
    # only selecting out roads that are sealed and fit the taxonomy of artificial surfaces
    OSM_roads_filtered = OSM_roads[OSM_roads['highway'].isin(['primary', 'primary_link', 'secondary', 'secondary_link', 'trunk', 'trunk_link'])]
    OSM_roads_AOI = OSM_roads.cx[xmin:xmax, ymin:ymax]
    OSM_roads_xr = xr_rasterize(gdf=OSM_roads_AOI, da=wofs_mask)


In [ ]:
# combine OSM xarrays
OSM_xr = xr.where((OSM_blds_xr == 1) | (OSM_airports_xr == 1) | (OSM_roads_xr == 1), 1, 0)

artific_urb_cat_ds = OSM_xr.to_dataset(
    name="artific_urb_cat")

In [ ]:
# plot output
artific_urb_cat_ds.artific_urb_cat.plot()

### 5. Natural Water / Artificial Water

NONE

### **Collect environmental variables into array for passing to classification system**

In [ ]:
variables_xarray_list = []
variables_xarray_list.append(vegetat_veg_cat_ds)
variables_xarray_list.append(aquatic_wat_cat_ds)
variables_xarray_list.append(cultman_agr_cat_ds)
variables_xarray_list.append(artific_urb_cat_ds)
# variables_xarray_list.append(artwatr_wat_cat_ds)

## <font color=blue>Level 3 classification</font>

**The LCCS classificaition is hierachial. The 8 classes are shown below**

| Class name | Code | Numeric code |
|----------------------------------|-----|-----|
| Cultivated Terrestrial Vegetated | A11 | 111 |
| Natural Terrestrial Vegetated | A12 | 112 |
| Cultivated Aquatic Vegetated | A23 | 123 |
| Natural Aquatic Vegetated | A24 | 124 |
| Artificial Surface | B15 | 215 |
| Natural Surface | B16 | 216 |
| Artificial Water | B27 | 227 |
| Natural Water | B28 | 228 |


In [ ]:
# Merge to a single dataframe
classification_data = xr.merge(variables_xarray_list)

In [ ]:
# Apply Level 3 classification using separate function. Works through in three stages
level1, level2, level3 = lccs_l3.classify_lccs_level3(classification_data)

In [ ]:
# Save classification values back to xarray
out_class_xarray = xr.Dataset(
    {"level1" : (classification_data["vegetat_veg_cat"].dims, level1),
     "level2" : (classification_data["vegetat_veg_cat"].dims, level2),
     "level3" : (classification_data["vegetat_veg_cat"].dims, level3)})
classification_data = xr.merge([classification_data, out_class_xarray])

In [ ]:
# Creating an array of non-valid bare surface because of the wofs' nan issue
classification_nan = ((classification_data.level3.where(
                            (classification_data.level3==216) & 
                            (wofs.frequency.isnull())))*0).fillna(1)

In [ ]:
# Filtering non-valid bare surface (i.e, due to NaN in WOFs) out of level 2 and level 3
# Level2 set to zero where WOFs is NaN (i.e., info on water/terrestrial in non-veg areas isn't valid)
classification_data["level2"] = classification_data.level2 * classification_nan

# Set Level3 to Level1 value where Level2 is zero
classification_data["level3"] = ((classification_data.level3.where((classification_data.level1==200) & (
                                    classification_data.level2==0))*0+200).fillna(0) + (
                                classification_data.level3.where(classification_data.level2!=0).fillna(0)))

## <font color=blue>Level 3 results</font>

In [ ]:
red, green, blue, alpha = lccs_l3.colour_lccs_level3(classification_data.level3.values)
img = np.dstack([red, green, blue, alpha])

# Rotate the image by 180 degrees - SHOULDN'T HAVE TO DO THIS BUT WAS NOT DISPLAYING PROPERLY
img = np.rot90(img, 2)

# Display the rotated image
plt.figure(figsize=(12, 10))
plt.imshow(img)
plt.show()


## <font color=blue>Create environmental layers for Level 4</font>

In [ ]:
# Convert level3 to Dataset and add name
level3_ds = classification_data.level3.to_dataset(name="level3")

### 1. Water state
<font color=red>**TODO:** could do this using wofs if we wanted </font>

### 2. Water persistence
<font color=red>**TODO:** could do this using wofs if we wanted </font>

### 3. Lifeform
Describes the detail of vegetated classes, separating woody from herbaceous  
0: Not applicable (such as in water areas)  
1: Woody (trees, shrubs)  
2: Herbaceous (grasses, forbs)

#### Sentinel1-based version

In [ ]:
# Open woodyarti tif file as xarray
# woody_s1_2020 = '../data/Woodyarti_30m_PNG.tif'
woody_s1_2020 = "s3://oa-bluecarbon-work-easi/livingearth-png/Woodyarti_30m_PNG.tif"
woody_s1_layer = rio_slurp_xarray(woody_s1_2020, gbox=vegetat_veg_cat_ds.geobox)

In [ ]:
woody_layer = (mangrove + woody_s1_layer)

In [ ]:
# Convert binary woodyarti layer to lifeform lccs classes
lifeform = woody_layer.where(woody_layer > 0)*0+1
lifeform = lifeform.fillna(2)
lifeform_veg_cat_ds = lifeform.to_dataset(name="lifeform_veg_cat").squeeze()

#### Landsat-based version

In [ ]:
# # Load woody cover fraction

# # Need to add any tranformations for the VP you're using
# # Get location of transformation
# transformation = "WCF"
# trans_loc = importlib.import_module(transformation)
# trans_class = transformation.split('.')[-1]

# DEFAULT_RESOLVER.register('transform', trans_class, getattr(trans_loc, trans_class) )

In [ ]:
# # see what's already in the resolver 
# DEFAULT_RESOLVER.lookup_table['transform']

In [ ]:
# # load WCF
# product = catalog['WCF']
# wcf = product.load(dc, **query)
# wcf_da = wcf.WCF

In [ ]:
# lifeform = wcf_da.copy()
# # threshold of woody and non woody vegetation
# lifeform.values = np.where(lifeform >= 0.2, 1, 2)

# # Convert to Dataset and add name
# lifeform_veg_cat_ds = lifeform.to_dataset(name="lifeform_veg_cat").squeeze()#.drop('time')

In [ ]:
# lifeform_veg_cat_ds.lifeform_veg_cat.plot()

### 4. Canopy cover
<font color=red>**TODO:** could do this using fractional cover if we wanted </font>

## <font color=blue>Level 4 classification</font>

In [ ]:
variables_xarray_list = []
variables_xarray_list.append(level3_ds)
# variables_xarray_list.append(waterstt_wat_cat_ds)
# variables_xarray_list.append(waterper_wat_cin_ds)
variables_xarray_list.append(lifeform_veg_cat_ds)
# variables_xarray_list.append(canopyco_veg_con_ds)

In [ ]:
# Merge to a single dataframe
l4_classification_data = xr.merge(variables_xarray_list)

In [ ]:
# Apply Level 4 classification
classification_array = lccs_l4.classify_lccs_level4(l4_classification_data)

In [ ]:
# Get Level 4 codes to apply colour scheme
l4_classification_codes = lccs_l4.get_lccs_level4_code(classification_array)
print(np.unique(l4_classification_codes))

In [ ]:
classification_level4 = (classification_array.level3*10.)+(
                       classification_array.lifeform_veg_cat_l4a)
print(np.unique(classification_level4))

## <font color=blue>Level 4 results</font>

In [ ]:
# # To check the results for level 3 use colour_lccs_level3 to get the colour scheme.
# # WIP: can't get plot to populate, problem with colour scheme maybe?
# pyplot.figure(figsize=(12, 10))
# red, green, blue, alpha = lccs_l4.colour_lccs_level4(l4_classification_codes)
# pyplot.imshow(numpy.dstack([red, green, blue, alpha]))

## <font color=blue>Select out blue carbon ecosystems (mangrove, saltmarsh, tidal woody area) from level 3 and 4 </font>

### 1. Mangrove ecosystem
- level 3 == 124
- lifeform == 1
- GMW == 1

In [ ]:
mangrove_class = (level3_ds.level3.where((classification_array.level3 == 124) & (
                    classification_array.lifeform_veg_cat_l4a == 1) & (
                    mangrove == 1))*0+1).fillna(0)

In [ ]:
mangrove_class.plot()

### 2. Tidal woody ecosystem
- level 3 == 124
- lifeform == 1
- GMW == 0

In [ ]:
tidal_woody_class = (level3_ds.level3.where((classification_array.level3 == 124) & (
                    classification_array.lifeform_veg_cat_l4a == 1) & (
                    mangrove != 1))*0+2).fillna(0)

In [ ]:
tidal_woody_class.plot()

### 3. Saltmarsh ecosystem
- level 3 == 124
- lifeform == 2

In [ ]:
saltmarsh_class = (level3_ds.level3.where((classification_array.level3 == 124) & (
                    classification_array.lifeform_veg_cat_l4a == 2))*0+3).fillna(0)

In [ ]:
saltmarsh_class.plot()

## <font color=blue>Blue carbon ecosystems</font>

In [ ]:
# combine
BCE = mangrove_class + saltmarsh_class + tidal_woody_class
BCE = BCE.where(BCE !=0, classification_level4)

In [ ]:
from matplotlib.colors import ListedColormap

# create a custom colormap with green, yellow, and brown colors
cmap = ListedColormap(["#ffffff", "#36a86d", "#1a8745", "#75e3a7", "#1b6924", "#88e673",
                       "#1a8745", "#60bd96", "#b1a9ba", "#e65a5a", "#c9a571", "#55b2e0"])

# specify the color levels corresponding to each value
levels = [0., 0.5, 1.5, 2.5, 3.5, 1121.5, 1122.5, 
          1241.5, 1242.5, 2000.5, 2150.5, 2160.5, 2200.5]

# create the plot with custom colors and color levels
# plot = BCE.plot(cmap=cmap, levels=levels, add_colorbar=False)
plot = BCE.plot(cmap=cmap, levels=levels, add_colorbar=False)

# add a legend
# legend_labels = {1: "Mangrove", 2: "Tidal woody", 3: "Saltmarsh"}
# handles = [plt.Rectangle((0,0),1,1, color=color) for color in cmap(np.arange(cmap.N))]
# labels = [legend_labels[i+1] for i in range(len(legend_labels))]
# plt.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc='upper left')

# change fig size and show the plot
fig = plt.gcf()
fig.set_size_inches(10, 8)
plt.show()

### **Save results to geotiff**

In [ ]:
# min_x = classification_data.coords['x'].min().values
# max_x = classification_data.coords['x'].max().values
# min_y = classification_data.coords['y'].min().values
# max_y = classification_data.coords['y'].max().values

# res_x = 30 
# res_y = -30 
# crs = 'EPSG:32755'
# # Write out
# output_rgb_file_name = "../data/level3_test.tif"
# out_file_transform = [res_x, 0, min_x, 0, res_y, max_y]
# output_x_size = int((max_x - min_x)/res_x)
# output_y_size = int((min_y - max_y)/res_y)

# # Write RGB colour scheme out
# rgb_dataset = rasterio.open(output_rgb_file_name, 'w', driver='GTiff',
#                             height=output_y_size, width=output_x_size,
#                             count=3, dtype=level3.dtype,
#                             crs=crs, transform=out_file_transform)
# rgb_dataset.write(red, 1)
# rgb_dataset.write(green, 2)
# rgb_dataset.write(blue, 3)
# rgb_dataset.close()

In [ ]:
client.close()

cluster.close()